## 二、调用 ChatGPT

## 1. ChatGPT

ChatGPT，发布于2022年11月，是目前火热出圈的大语言模型（Large Language Model，LLM）的代表产品。在2022年底，也正是 ChatGPT 的惊人表现引发了 LLM 的热潮。时至目前，由 OpenAI 发布的 ChatGPT-4 仍然是 LLM 性能上限的代表，ChatGPT 也仍然是目前使用人数最多、使用热度最大、最具发展潜力的 LLM 产品。事实上，在圈外人看来，ChatGPT 即是 LLM 的代称。

OpenAI 除发布了免费的 Web 端产品外，也提供了多种 ChatGPT API，支持开发者通过 Python 或 Request 请求来调用 ChatGPT，向自己的服务中嵌入 LLM 的强大能力。可选择的主要模型包括 ChatGPT-3.5 和 ChatGPT-4，但每个模型也存在多个上下文版本，例如 ChatGPT-3.5 就有最原始的 4K 上下文长度的模型，也有 16K 上下文长度的模型 gpt-turbo-16k-0613。

在本章节，我们将主要讲述两种通过 Python 代码调用 ChatGPT API 的方法：直接调用 OpenAI 的原生接口，或是基于 LangChain 调用 ChatGPT API。

## 2. 获取并配置 OpenAI API Key

OpenAI API 服务是付费的，每一个开发者都需要首先获取并配置 OpenAI API Key，才能在自己构建的应用中访问 ChatGPT。我们将在这部分简述如何获取并配置 OpenAI API Key。

在获取OpenAI API key之前我们需要[openai官网](https://openai.com/)中注册一个账号。这里假设我们已经有了openai账号，先在[openai官网](https://openai.com/)登录，登录后如下图所示：

<p align="center">
  <img src="../../figures/openai-choose.png" width="1000" alt="openai官网登录后选择API">
</p>

我们选择`API`，然后点击右上角的头像，选择`View API keys`，如下图所示：

<p align="center">
  <img src="../../figures/openai-get-key.png" width="1000" alt="openai获取key">
</p>

点击`Create new secret key`按钮创建OpenAI API key，我们将创建好的OpenAI API key复制以此形式`OPENAI_API_KEY="sk-..."`保存到`.env`文件中，并将`.env`文件保存在项目根目录下。

下面是读取`.env`文件的代码

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 如果你需要通过代理端口访问，你需要如下配置
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
os.environ["HTTP_PROXY"] = 'http://127.0.0.1:7890'

# 获取环境变量 OPENAI_API_KEY
openai.api_key = os.environ['OPENAI_API_KEY']

## 2. 调用 OpenAI 原生接口

调用 ChatGPT 需要使用 [ChatCompletion API](https://platform.openai.com/docs/api-reference/chat)，该 API 提供了 ChatGPT 系列模型的调用，包括 ChatGPT-3.5，ChatGPT-4 等。

ChatCompletion API 调用方法如下：

In [4]:
import openai
# 导入所需库
# 注意，此处我们假设你已根据上文配置了 OpenAI API Key，如没有将访问失败
completion = openai.ChatCompletion.create(
    # 创建一个 ChatCompletion
    # 调用模型：ChatGPT-3.5
    model="gpt-3.5-turbo",
    # message 是你的 prompt
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello!"}
    ]
)


调用该 API 会返回一个 ChatCompletion 对象，其中包括了回答文本、创建时间、ID等属性。我们一般需要的是回答文本，也就是回答对象中的 content 信息。

In [5]:
completion

<OpenAIObject chat.completion id=chatcmpl-80QUFny7lXqOcfu5CZMRYhgXqUCv0 at 0x7f1fbc0bd770> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Hello! How can I assist you today?",
        "role": "assistant"
      }
    }
  ],
  "created": 1695112507,
  "id": "chatcmpl-80QUFny7lXqOcfu5CZMRYhgXqUCv0",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 9,
    "prompt_tokens": 19,
    "total_tokens": 28
  }
}

In [6]:
print(completion["choices"][0]["message"]["content"])

Hello! How can I assist you today?


此处我们详细介绍调用 API 常会用到的几个参数：

    · model，即调用的模型，一般取值包括“gpt-3.5-turbo”（ChatGPT-3.5）、“gpt-3.5-16k-0613”（ChatGPT-3.5 16K 版本）、“gpt-4”（ChatGPT-4）。注意，不同模型的成本是不一样的。

    · message，即我们的 prompt。ChatCompletion 的 message 需要传入一个列表，列表中包括多个不同角色的 prompt。我们可以选择的角色一般包括 system：即前文中提到的 system prompt；user：用户输入的 prompt；assitance：助手，一般是模型历史回复，作为给模型参考的示例。

    · temperature，温度。即前文中提到的 Temperature 系数。

OpenAI 提供了充分的自定义空间，支持我们通过自定义 prompt 来提升模型效果，但我们一般不需要使用到 system prompt 与 assistance prompt，因此我们会给出如下一个函数，对 OpenAI API 做了一定封装，支持我们直接传入 Prompt 并获得模型的输出：

In [7]:
# 一个封装 OpenAI 接口的函数，参数为 Prompt，返回对应结果
def get_completion(prompt, model="gpt-3.5-turbo", temperature = 0):
    '''
    prompt: 对应的提示词
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    '''
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # 模型输出的温度系数，控制输出的随机程度
    )
    # 调用 OpenAI 的 ChatCompletion 接口
    return response.choices[0].message["content"]


在上述函数中，我们封装了 messages 的细节，仅使用 user prompt 来实现调用。事实上，在简单场景中，该函数完全足够使用。

## 3. 基于 LangChain 调用 ChatGPT

LangChain 提供了对于多种大模型的封装，基于 LangChain 的接口可以便捷地调用 ChatGPT 并将其集合在以 LangChain 为基础框架搭建的个人应用中。我们在此简述如何使用 LangChain 接口来调用 ChatGPT。

注意，基于 LangChain 接口调用 ChatGPT 同样需要配置你的个人密钥，配置方法同上。